In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 93.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 103.6 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import shutil
import sys  
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from bs4 import BeautifulSoup
import re
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from tqdm.notebook import tqdm

In [ ]:
data_final =  '/content/drive/MyDrive/Final Project/Final Data/final_data'
data_final = pd.read_csv(data_final)

In [ ]:
df = data_final[['content','label']]

In [ ]:
import json
with open('/content/drive/MyDrive/Final Project/Data Preprocessing/Part 1/data_matrix_3D_with_no_stopwords_after_unite_para.json', 'r') as file:
    data = json.load(file)

In [ ]:
import random


In [ ]:
df['content_v1'] = data

<ipython-input-11-0ec29629a76c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['content_v1'] = data


In [ ]:
target_list = ['0', '1', '2']

In [ ]:
# Create new columns for each category and assign values
df['0'] = df['label'].apply(lambda x: 1 if x == 0 else 0)
df['1'] = df['label'].apply(lambda x: 1 if x == 1 else 0)
df['2'] = df['label'].apply(lambda x: 1 if x == 2 else 0)

# Remove the original column
df.drop('label', axis=1, inplace=True)



In [ ]:
df.drop('content',axis=1,inplace=True)


In [ ]:
df = df.rename(columns={'content_v1':'content'})

In [ ]:
# hyperparameters
MAX_LEN = 450
TRAIN_BATCH_SIZE = 64
VALID_BATCH_SIZE = 64
EPOCHS = 2
LEARNING_RATE = 1e-04

In [ ]:
from transformers import BertTokenizer, Trainer, BertForSequenceClassification, TrainingArguments


In [ ]:
model = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-pretrain',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-pretrain')

In [ ]:
# Split to train,validation,test
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df ,test_size=0.2, random_state=42)

In [ ]:
train_df, test_df =train_test_split(train_df ,test_size=0.25, random_state=42)

In [ ]:
train_df = train_df.reset_index(drop=False)
train_df = train_df.sort_values('index')
train_df = train_df.drop('index',axis=1).reset_index(drop=True)


In [ ]:
val_df = val_df.reset_index(drop=False)
val_df = val_df.sort_values('index')
val_df = val_df.drop('index',axis=1).reset_index(drop=True)


In [ ]:
# This function takes from each sample x (x resmble the size of the batch in the model) paragraph of 450 length
def shuffle(lst,batch_size):

    return lst[:batch_size]

In [ ]:
# This function got df and return new df which contain for each sample X ( as many we decide the batch size is) sub sample of their subtext for choose randomly, for example the 32 first rows in the new df will contain the 32 subtext of the
# first sample and etc..
def smaller(df,batch_size):
    # Iterate over each row in the original DataFrame
    content_lst = []
    lst0 = []
    lst1 = []
    lst2 =[]
    for index, row in df.iterrows():
      temp = shuffle(row['content'],batch_size)
      if(len(temp)) < batch_size:
        continue
      else:
        for i in range(0,len(temp)):
          content_lst.append(temp[i])
          lst0.append(row['0'])
          lst1.append(row['1'])
          lst2.append(row['2'])
    new_df = np.array([content_lst,lst0,lst1,lst2])
    new_df = pd.DataFrame(new_df.T,columns = ['content','0','1','2'])
    return new_df

In [ ]:
# Here we taking each sample and divide the sample to 64 parts
train_df = smaller(train_df,64)

In [ ]:
val_df = smaller(val_df,64)

In [ ]:
val_df.shape

(241856, 4)

In [ ]:
train_df['0'] = train_df['0'].astype('int64')
train_df['1'] = train_df['1'].astype('int64')
train_df['2'] = train_df['2'].astype('int64')
val_df['0'] = val_df['0'].astype('int64')
val_df['1'] = val_df['1'].astype('int64')
val_df['2'] = val_df['2'].astype('int64')

In [ ]:
# This is custom dataset, for loding throughout pythorch loadr
# This data set return for each sample a dictinary which include his tokenization using the tokenization of finbert
class CustomDataset(torch.utils.data.Dataset):

    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.df = df
        self.title = df['content']
        self.targets = self.df[target_list].values
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        title = str(self.title[index])
        title = " ".join(title.split())

        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'targets': torch.FloatTensor(self.targets[index])
        }
          

In [ ]:
train_dataset = CustomDataset(train_df, tokenizer, MAX_LEN)
valid_dataset = CustomDataset(val_df, tokenizer, MAX_LEN)

In [ ]:
train_data_loader = torch.utils.data.DataLoader(train_dataset, 
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True,
    num_workers=0
)

val_data_loader = torch.utils.data.DataLoader(valid_dataset, 
    batch_size=VALID_BATCH_SIZE,
    shuffle=False,
    num_workers=0
)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into       
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss 
    return model, optimizer, checkpoint['epoch'], valid_loss_min.item()

def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)

In [ ]:
# Building the model
# The model include the finbert model pre traind
# 3 convultional layers
# 2 FF layers
class FinBertSequenceClassifier(torch.nn.Module):
    def __init__(self, num_labels=3):
        super(FinBertSequenceClassifier, self).__init__()
        self.device = device
        self.bert_model = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-pretrain',num_labels=3)
        self.conv1 = nn.Conv1d(in_channels=64, out_channels=256, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(in_channels=256, out_channels=128, kernel_size=3, padding=1)
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(3, 128)
        self.conv3 = nn.Conv1d(in_channels=128, out_channels=64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128,3)
        self.weight_decay = 0.02
    
    def forward(self, input_ids, attn_mask, token_type_ids):
        output = self.bert_model(
            input_ids, 
            attention_mask=attn_mask, 
            token_type_ids=token_type_ids
        ).logits
        x = self.conv1(output)
        x = nn.functional.relu(x)
        x = self.dropout(x)
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = self.conv3(x)
        x = nn.functional.relu(x)
        x = self.dropout(x)
        x = self.fc(x)
        x = nn.functional.relu(x)
        x = self.dropout(x)
        logits = self.fc1(x)
        output = nn.functional.softmax(logits, dim=1)
        return output

    




In [ ]:
model = FinBertSequenceClassifier()
model.to(device)

Some weights of the model checkpoint at yiyanghkust/finbert-pretrain were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

FinBertSequenceClassifier(
  (bert_model): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30873, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_featu

In [ ]:
import os

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
def l2_regularization(model, weight_decay):
    l2_reg = torch.tensor(0.0).to(model.device)
    for name, param in model.named_parameters():
        if 'bias' not in name:
            l2_reg += torch.norm(param, p=2)
    return l2_reg * weight_decay

In [ ]:
def loss_function(outputs, targets, model, weight_decay):
    loss = nn.CrossEntropyLoss()(outputs, targets)
    loss += l2_regularization(model, weight_decay)
    return loss

optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
# this function take all the batches output and return the average output of all the batch
# we do this because each batch resamble one sample (each origanl sample divided to 64 parts)
def get_out_final_output(outputs):

  sum = outputs.sum(dim=0) / len(outputs)
  return sum


In [ ]:
from sklearn.metrics import accuracy_score


In [ ]:
def train_model(n_epochs, training_loader, validation_loader, model, 
                optimizer, checkpoint_path, best_model_path,device):
   
  # initialize tracker for minimum validation loss
  valid_loss_min = np.Inf
   
 
  for epoch in range(1, n_epochs+1):
    train_loss = 0
    valid_loss = 0
    val_correct = 0
    val_total = 0
    model.train()
    print('############# Epoch {}: Training Start   #############'.format(epoch))
    for batch_idx, data in enumerate(training_loader):
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)
        outputs = model(ids, mask, token_type_ids)
        outputs =  get_out_final_output(outputs)
        optimizer.zero_grad()
        outputs = get_out_final_output(outputs)
        targets = targets[0]
        loss = loss_function(outputs, targets,model,0.01)
        if batch_idx%2500==0:
           print(f'Epoch: {epoch}, Training Loss:  {loss.item()}', 'Batch idx', {batch_idx})
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #print('before loss data in training', loss.item(), train_loss)
        train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.item() - train_loss))
        #print('after loss data in training', loss.item(), train_loss)
    
    print('############# Epoch {}: Training End     #############'.format(epoch))
    
    print('############# Epoch {}: Validation Start   #############'.format(epoch))
    ######################    
    # validate the model #
    ######################
 
    model.eval()
   
    with torch.no_grad():
      val_targets1 = []
      val_outputs1 = []
      for batch_idx, data in enumerate(validation_loader, 0):
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            outputs =  get_out_final_output(outputs)
            targets = targets[0]
            loss = loss_function(outputs, targets,model,0.01)
            temp = outputs.tolist()
            max = np.argmax(temp)
            new_lst = np.array([1 if i == max else 0 for i in range(len(temp))])
            val_outputs1.append(new_lst)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.item() - valid_loss))
            val_targets1.append(np.array(targets.tolist()))


      print('############# Epoch {}: Validation End     #############'.format(epoch))
      # calculate average losses
      #print('before cal avg train loss', train_loss)
      train_loss = train_loss/len(training_loader)
      valid_loss = valid_loss/len(validation_loader)

            # caculate accuracy
      val_acc = accuracy_score(val_targets1,val_outputs1)
      # print training/validation statistics 
      print("Epoch: {}, Training Loss: {:.6f}, Validation Loss: {:.6f}, Validation Accuracy: {:.6f}".format(

            epoch, 
            train_loss,
            valid_loss,
            val_acc
            ))
      
      # create checkpoint variable and add important data
      checkpoint = {
            'epoch': epoch + 1,
            'valid_loss_min': valid_loss,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
      }
        
        # save checkpoint
      save_ckp(checkpoint, False, checkpoint_path, best_model_path)
        
      ## TODO: save the model if validation loss has decreased
      if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
        # save checkpoint as best model
        save_ckp(checkpoint, True, checkpoint_path, best_model_path)
        valid_loss_min = valid_loss

    print('############# Epoch {}  Done   #############\n'.format(epoch))

  return model

In [ ]:
ckpt_path = "/content/drive/MyDrive/Final Project/FinBert Model/finbert"
best_model_path = "/content/drive/MyDrive/Final Project/FinBert Model/finbert_best.pt"

In [ ]:
trained_model = train_model(EPOCHS, train_data_loader, val_data_loader, model, optimizer, ckpt_path, best_model_path,device)

############# Epoch 1: Training Start   #############
Epoch: 1, Training Loss:  30.893798828125 Batch idx {0}
Epoch: 1, Training Loss:  5.993905544281006 Batch idx {2500}
Epoch: 1, Training Loss:  4.1845574378967285 Batch idx {5000}
Epoch: 1, Training Loss:  2.4319796562194824 Batch idx {7500}
Epoch: 1, Training Loss:  0.8534832000732422 Batch idx {10000}
############# Epoch 1: Training End     #############
############# Epoch 1: Validation Start   #############
############# Epoch 1: Validation End     #############
Epoch: 1, Training Loss: 0.000387, Validation Loss: 0.000207, Validation Accuracy: 0.410426
Validation loss decreased (inf --> 0.000207).  Saving model ...
############# Epoch 1  Done   #############

############# Epoch 2: Training Start   #############
Epoch: 2, Training Loss:  0.7691027522087097 Batch idx {0}
Epoch: 2, Training Loss:  0.7246585488319397 Batch idx {2500}
Epoch: 2, Training Loss:  0.7572054266929626 Batch idx {5000}
Epoch: 2, Training Loss:  0.8659952878

In [ ]:
test_df = test_df.reset_index(drop=False)
test_df = test_df.sort_values('index')
test_df = test_df.drop('index',axis=1).reset_index(drop=True)


In [ ]:
# Here we taking each sample and divide the sample to 64 parts

test_df = smaller(test_df,64)

In [ ]:
test_df['0'] = test_df['0'].astype('int64')
test_df['1'] = test_df['1'].astype('int64')
test_df['2'] = test_df['2'].astype('int64')

In [ ]:
test_dataset = CustomDataset(test_df,tokenizer,MAX_LEN)
batch_size = 64

In [ ]:
test_data_loader = torch.utils.data.DataLoader(test_dataset, 
    batch_size=batch_size,
    shuffle=False,
    num_workers=0
)

In [ ]:
# this fuction calculate the accuracy of the test
# we divided each sample of the test to 64 parts and put them as a batch in the model
# each batch we calcualte the average output for example for 2 softmax outputs [0.2,0.1,0.7] & [0.8,0.1,0.1] the average output will be [0.5,0.1,0.4]
# because each output in the batch have the same label because their are part of the original sample we wiil take the label of the first sub sample in the batch
output_lst = []
target_lst =[]
model.eval()
with torch.no_grad():
    for batch_idx, data in enumerate(test_data_loader, 0):
      ids = data['input_ids'].to(device, dtype = torch.long)
      mask = data['attention_mask'].to(device, dtype = torch.long)
      token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
      targets = data['targets'].to(device, dtype = torch.float)
      outputs = model(ids, mask, token_type_ids)
      outputs =  get_out_final_output(outputs)
      temp = outputs.tolist()
      max = np.argmax(temp)
      new_lst = np.array([1 if i == max else 0 for i in range(len(temp))])
      output_lst.append(new_lst)
      targets = targets[0]
      target_lst.append(np.array(targets.tolist()))

In [ ]:
test_acc = accuracy_score(target_lst,output_lst)
print('The  Accuracy is :', test_acc)

The  Accuracy is : 0.4251258945136496
